[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/storopoli/ciencia-de-dados/master?filepath=notebooks%2FAula_12_Regressao_Logistica.ipynb)
<br>
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/storopoli/ciencia-de-dados/blob/master/notebooks/Aula_12_Regressao_Logistica.ipynb)

# Regressão Logística

**Objetivos**: Aprender o que é Regressão Logística e Regressão Softmax usando a biblioteca `Scikit-Learn`. Introduzir o aluno aos problemas de classificação de aprendizagem de máquina.

## Defininição - Regressão Logística

> Uma regressão logística se comporta exatamente como um modelo linear: faz uma predição simplesmente computando uma soma ponderada dos atributos (*features*), mais uma constante chamada viés (*bias*), também chamado de constante (*intercept*). Porém ao invés de retornar um valor contínuo, como a regressão linear, retorna a função logística desse valor.

$$\operatorname{Logística}(x) = \frac{1}{1 + e^{(-x)}}$$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
x = np.linspace(-10, 10, 100)
sig = 1 / (1 + np.exp(-x))
plt.plot(x, sig, "b-", linewidth=2, label=r"$\sigma(x) = \frac{1}{1 + e^{-x}}$")
plt.xlabel("x")
plt.legend(loc="upper left", fontsize=15)
plt.show()

$$ \operatorname{Linear} = \theta_0 + \theta_1 x_1 + \theta_2 x_2 + \dots + \theta_n x_n$$

$\operatorname{Linear}$ - regressão linear

$\theta$ - parâmetro do modelo

$n$ - número de atributos (*features*)

$x_i$ - o valor do *inésimo* atributo (*feature*)

$ \hat{p} = \sigma(\operatorname{Linear}) = \frac{1}{1 + e^{-\operatorname{Linear}}}$

$\hat{p}$ - probabilidade prevista da observação ser 1

$\hat{y}=\left\{\begin{array}{ll} 0 & \text { se } \hat{p} < 0.5 \\ 1 & \text { se } \hat{p} \geq 0.5 \end{array}\right.$

### Exemplo

$\mathrm{Previsão~de~Morte} = \sigma \big(-10 + 10\times \mathrm{cancer} + 12 \times \mathrm{diabetes} + 8 \times \mathrm{obesidade} \big)$

## Métricas de Desempenho de uma Regressão Logística

### MSE (?)
$$MSE = \frac{1}{m}\Sigma_{i=1}^{m}{(\hat{y}_i - y_i)^2}$$
Nossa predição agora é não-linear (devido à transformação logística). Se nós elevarmos essa predição ao quadrado (como fazemos em MSE) resulta em uma função não convexa e com muitos local minima, portanto, inviabilizando o Método do Gradiente Descendente.

### *Log Loss* 

Se divide em duas métricas de desempenho

$$Log~Loss=\left\{\begin{array}{ll} -\log(\hat{p}) & \text { se } y = 1 \\ -\log(1 - \hat{p}) & \text { se } y = 0 \end{array}\right.$$

Faz sentido porque:

* $- \log(\hat{p})$ se torna grande quando $\hat{p}$ se aproxima de 0 - Erro vai ser grande quando o modelo prevê $\hat{p} \approx 0$ mas $y = 1$
* $- \log(1 - \hat{p})$ se torna grande quando $1- \hat{p}$ se aproxima de 0 - Erro vai ser grande quando o modelo prevê $\hat{p} \approx 1$ mas $y = 0$

In [ ]:
fig,ax = plt.subplots(1,2, sharey=True)

x = np.linspace(0.001, 0.999, 100)  # evitar erros pq log(0) é inf e log(1) é 0
ax[0].plot(x, -np.log(x))
ax[0].set_title('Se $y=1$')
ax[0].set_xlabel('$\hat{p}$')
ax[0].set_ylabel('Erro')

ax[1].plot(x, -np.log(1-x))
ax[1].set_title('Se $y=0$')
ax[1].set_xlabel('$\hat{p}$')
ax[1].set_ylabel('Erro')

plt.show()

### Para todo o dataset

$$Log~Loss = \frac{1}{m} \sum_{i=1}^{m} \big[y^{(i)} \log (\hat{p}^{(i)}) + (1+y^{(i)}) \log (1-\hat{p}^{(i)}) \big]$$

## Exemplo com o Dataset Titanic

Contém 891 passageiros reais do Titanic que afundou em 15/04/1912 matando 1502 de 2224 passageiros e tripulação que estavam a bordo.

* `survived`: *dummy* `0` ou `1` 
* `pclass`: Classe do Passageiro
    - `1`: Primeira Classe
    - `2`: Segunda Classe
    - `3`: Terceira Classe
* `sex`: Sexo `male` ou `female`
* `age`: Idade
* `sibsp`: Número de Irmãos (*Siblings*) e Esposas (*spouse*) a bordo
* `parch`: Número de pais/filhos a bordo
* `fare`: Valor pago pela passagem em libras
* `embarked`: Porto que embarcou
    - `C`: Cherbourg
    - `Q`: Queenstown
    - `S`: Southampton)
* `class`: Mesmo que `pclass` só que em texto
* `adult_male`: *dummy* para `age > 16` e `sex == 'male'`
* `deck`: Qual deck a cabine do passageiro se situava
* `alive`: Mesmo que survived só que com `yes` ou `no`
* `alone`: *dummy* para se viajava sozinho

<img src="images/titanic.png" alt="titanic" style="width: 500px;"/>

In [ ]:
import seaborn as sns
import pandas as pd
titanic = sns.load_dataset('titanic')

feature_names = ['pclass', 'female', 'age', 'fare']
titanic['female'] = titanic['sex'].map({'male': 0, 'female': 1})
titanic.dropna(subset=feature_names, inplace=True)  #891 para 714

X = titanic[feature_names].to_numpy()
y = titanic['survived'].to_numpy()

In [ ]:
X  # Numpy Array apenas com números

In [ ]:
y  # Numpy Array 1-D com 0 ou 1

In [ ]:
print('Nomes dos Atributos: ', feature_names, '\n')
print('Tamanho de X: ', X.shape, '\n')
print('Tamanho de y: ', y.shape, '\n')

### Quebrando dataset em `train` e `test`

Usar a função do Scikit-Learn [`sklearn.model_selection.train_test_split()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

#### Argumentos:

* matriz a ser dividida - `X` ou `y`
* `test_size` - `float` ou `int` do tamanho do dataset de teste (padrão $0.25$)
* `train_size` - padrão `1 - test_size`
* `random_state` - `int` - seed do gerador de número randômicos (replicabilidade)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=123)

In [ ]:
print('Tamanho de X_train: ', X_train.shape, '\n')
print('Tamanho de X_test: ', X_test.shape, '\n')
print('Tamanho de y_train: ', y_train.shape, '\n')
print('Tamanho de y_test: ', y_test.shape, '\n')

### Regressão Logística
Usar a função do Scikit-Learn [`sklearn.linear_model.SGDClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html)

#### Argumentos:
* `loss` - `str`
    * Log Loss - `'log'`
* `max_iter` - `int` - Número máximo de iterações do *Gradient Descent*
* `tol` - Tolerância - Critério de parada de treino
* `random_state` - `int` - seed do gerador de número randômicos (replicabilidade)
* `eta0` - `float` - Taxa de aprendizagem inicial
    * padrão `0.01`
* `learning_rate` - `str` - Taxa de aprendizagem
    * Constante - `'constant'`
    * Adapatativa - `'adaptive'`
* `n_iter_no_change` - `int` - Somente se usar Taxa de Aprendizagem Adaptativa

#### Retorna:
* Objeto `estimator` do Scikit-Learn

In [ ]:
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier(loss='log', learning_rate='constant', max_iter=10,
                   eta0=0.0001, verbose=1, tol=None, random_state=123)

### Classe `Estimators`

* `.fit()` - Treina o Modelo
    * `X`
    * `y`
* `.predict()` - Gera predições do modelo
    * `X`
* `.coef_` - Retorna os coeficientes do modelo ($\theta_i$)
* `.intercept_` - Retorna o viés/constante (*bias/intercept*) do modelo ($\theta_0$)

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
# Coeficientes do modelo
for feature, coef in zip(feature_names, clf.coef_[0].tolist()):
    print(f"{feature}: {round(coef,3)}")

# Constante do modelo
print(f"Constante: {clf.intercept_}")

### Acurácia do Modelo
Usar a função do Scikit-Learn [`sklearn.metrics.accuracy_score()`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html)

Retorna um score de acurácia `float` entre $0$ e $1$

#### Argumentos
* `y_true`: Classes Verdadeiras
    * 2 classes: vetor (1-D)
    * Mais que 2 classes: matriz (2-D)
* `y_pred`: Classes Previstas pelo Modelo
    * 2 classes: vetor (1-D)
    * Mais que 2 classes: matriz (2-D)

In [ ]:
from sklearn.metrics import accuracy_score

y_train_true = y_train
y_train_pred = clf.predict(X_train)
y_test_true = y_test
y_test_pred = clf.predict(X_test)


print(f"Acurácia de Treino: {round(accuracy_score(y_train_true, y_train_pred), 2)}")
print('\n ---------------------------\n')
print(f"Acurácia de Teste: {round(accuracy_score(y_test_true, y_test_pred), 2)}")

## Expandindo para $k$ classes

Podemos usar Regressão Logística para mais de uma duas classes ($k > 2$)

### Função Softmax

Dado um vetor $\mathbf{x}$, a função Softmax computa um score $s_k(\mathbf{x})$ para cada classe $k$, então estima a probabilidade de cada classe.

$$\hat{p}_k = \sigma(\mathbf{s}(\mathbf{x}))_k = \frac{e^{s_k(\mathbf{x})}}{\sum_{j=1}^K e^{s_j(\mathbf{x})}}$$

* $K$ - número de classes
* $\mathbf{s}(\mathbf{x})$ - vetor contendo todos os scores de cada classe de uma observação $x$
* $\sigma(\mathbf{s}(\mathbf{x}))_k$ - a probabilidade estimada ($\hat{p}_k$) que a obsevação $x$ pertence a classe $k$, dado os scores de cada classe para $x$

### Regressão Softmax

Que nem a Regressão Logística, a Regressão Softmax prevê a classe com maior probabilidade estimada (que é simplesmente a classe com o maior score) 

$$\hat{y} = \max_k \sigma(\mathbf{s}(\mathbf{x}))_k = \max_k s_k(\mathbf{x})$$

### Regressão Softmax (Regressão Logística Multinomial)
Usar a função do Scikit-Learn [`sklearn.linear_model.Logistic Regression()`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)

Não possui opções de controle da taxa de aprendizagem do Método do Gradiente Descendente

#### Argumentos:
* `multi_class` - `str`
    * `'ovr'` - Binário ($k=2$)
    * `'multinomial'` - Multiclasses ($k > 2$)
    * `'auto'` - se baseia nas dimensões da `array` `y` que é passada no `.fit()`
* `max_iter` - `int` - Número máximo de iterações do *Gradient Descent*
* `tol` - Tolerância - Critério de parada de treino
* `random_state` - `int` - seed do gerador de número randômicos (replicabilidade)

#### Retorna:
* Objeto `estimator` do Scikit-Learn

#### O que muda da Regressão Logística?
Agora o meu $y$ é uma matriz (2-D) e não um vetor (1-D)

### Métrica de Desempenho de uma Regressão Softmax

**Cross Entropy**: Estender a Log Loss para mais que duas classes

$$Cross~Entropy = -\frac{1}{m} \sum_{i=1}^{m} \sum_{k=1}^K y_k^{(i)} \log (\hat{p}_k^{(i)})$$

* $y_k^{(i)}$ é a probabilidade alvo que a observação $i$ pertence à classe $k$. De maneira geral, ou é $1$ ou $0$, dependendo de qual classe $i$ pertence.

![mapa-conceitual](https://github.com/storopoli/ciencia-de-dados/raw/master/Mapas%20Conceituais/13%20-%20Regressao%20Logistica.png)

## Atividade - Regressão com o dataset [Iris](https://scikit-learn.org/stable/datasets/index.html#iris-dataset)

Edgar Anderson coletou os dados para quantificar a variação morfológica das flores de íris de três espécies relacionadas.

O conjunto de dados consiste em 50 amostras de cada uma das três espécies de Iris (Setosa, Virginica e Versicolor). Quatro características foram medidas em cada amostra (cm):

* $N = 150$
* Atributos: 4
    * `sepal length (cm)` - Cumprimento da Sépala
    * `sepal width (cm)` - Largura da Sépala
    * `petal length (cm)` - Cumprimento da Pétala
    * `petal width (cm)` - Largua da Sépala
* Variável dependente: Tipo de espécie de Iris
    * `0` - Setosa
    * `1` - Virginica
    * `2` - Versicolor 

* Achar a acurácia do modelo e os respectivos coeficientes dos atributos ($\theta_i$) e viés/constante ($\theta_0$)

>Obs: usar `test_size = 0.25` e `random_state = 123`

<img src="https://thegoodpython.com/assets/images/iris-species.png" alt="iris-sepals-petals" style="width: 600px;"/>

In [ ]:
from sklearn.datasets import load_iris

iris = load_iris()
X = iris['data']
y = iris['target']

In [ ]:
print('Nomes dos Atributos: ', iris['feature_names'], '\n')
print('Tamanho de X: ', X.shape, '\n')
print('Tamanho de y: ', y.shape, '\n')

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=123)

In [ ]:
print('Tamanho de X_train: ', X_train.shape, '\n')
print('Tamanho de X_test: ', X_test.shape, '\n')
print('Tamanho de y_train: ', y_train.shape, '\n')
print('Tamanho de y_test: ', y_test.shape, '\n')

In [ ]:
from sklearn.linear_model import LogisticRegression